# Implemented image manipulation ideas
### `apply_blur`
The function takes a NumPy array representing an image as input. It converts the array to a PIL Image, applies a Box Blur filter with a radius of 10 to the image using the filter() method, and then converts the modified PIL Image back to a NumPy array before returning the result.

In [ ]:
def apply_blur(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.filter(ImageFilter.BoxBlur(10))
    return np.array(pilImage)

### `create_rotate_image`

In [ ]:
def create_rotate_image(angle: int):
    def rotate_image(image: np.ndarray):
        pilImage = Image.fromarray(image)
        pilImage = pilImage.rotate(angle=angle)
        return np.array(pilImage)
    return rotate_image

### `flip_image_horizontally`
The function takes a NumPy array representing an image as input, converts it to a PIL Image, flips the image horizontally using the transpose() method with the specified transformation (FLIP_LEFT_RIGHT), and then converts the horizontally flipped PIL Image back to a NumPy array before returning the result.

In [ ]:
def flip_image_horizontally(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.transpose(Image.FLIP_LEFT_RIGHT)
    return np.array(pilImage)

### `change_to_grayscale`
The function accepts a NumPy array representing an image as input, transforms it into a PIL Image, converts the color image to grayscale using the convert() method with the 'L' mode, and then converts the resulting grayscale PIL Image back to a NumPy array before returning the processed image.

In [ ]:
def change_to_grayscale(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.convert('L')
    return np.array(pilImage)

### `apply_gauss_noise`
The function generates Gaussian noise with a mean of 50 and a standard deviation of 10 using NumPy's random.normal() function. This noise is created to match the shape of the input image. The generated noise is then added to the original image using the OpenCV add() function, resulting in an image with applied Gaussian noise.

In [ ]:
def apply_gauss_noise(image: np.ndarray):
    gauss = np.random.normal(50, 10, image.shape).astype('uint8')
    return cv2.add(image, gauss)

### `blur_edges`

In [ ]:
def blur_edges(image: np.ndarray):
    # edge detection
    grey_scale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    horizontalEdgeImage = convolve2d(grey_scale_image, np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]), mode='same', boundary='symm')
    verticalEdgeImage = convolve2d(grey_scale_image, np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]), mode='same', boundary='symm')
    plt.matshow(horizontalEdgeImage, cmap='gray')
    plt.show()
    plt.matshow(verticalEdgeImage, cmap='gray')
    plt.show()
    edge_image = pow(pow(horizontalEdgeImage, 2) + pow(verticalEdgeImage, 2), 0.5)
    plt.matshow(edge_image, cmap='gray')
    plt.show()

    plt.matshow(np.abs(edge_image), cmap='gray')
    plt.show()

    print(edge_image.shape)
    edge_image_all_colors = np.tile(edge_image, (1, 3))

    print(edge_image_all_colors.shape)
    edge_image_all_colors_reshaped = edge_image_all_colors.reshape(edge_image.shape[0], edge_image.shape[1], 3)

    print(edge_image_all_colors_reshaped.shape)
    image_edge_blur = image + ((edge_image_all_colors_reshaped - image) * 0.2).astype('uint8')
    plt.matshow(image_edge_blur)
    plt.show()

    laplacian = cv2.Laplacian(grey_scale_image, cv2.CV_64F)
    plt.matshow(laplacian, cmap='gray')
    plt.show()

    plt.matshow(np.abs(laplacian), cmap='gray')
    plt.show()

    return image_edge_blur

### `create_add_black_squares`
The outer function, `create_add_black_squares`, takes two integer parameters, `square_size` and `number_of_squares`, and returns an inner function `add_black_squares`. When this inner function is called with a NumPy array representing an image, it iteratively adds black squares to the image. The size and number of squares are determined by the parameters provided during the creation of the outer function. The positions of the black squares are randomly generated within the image dimensions, and the pixel values within the specified square regions are set to 0 (black). The resulting image is then adjusted to ensure pixel values remain within the valid 0-255 range before being returned.

In [ ]:
def create_add_black_squares(square_size: int, number_of_squares: int):
    def add_black_squares(image: np.ndarray):
        for _ in range(number_of_squares):
            x = np.random.randint(0, image.shape[0] - square_size)
            y = np.random.randint(0, image.shape[1] - square_size)
            image[x:x + square_size, y:y + square_size] = 0  # 0 because squares are black

        # Ensure the image values stay within 0-255 range
        return np.clip(image, 0, 255).astype(np.uint8)

    return add_black_squares

### `apply_bilateral_filter`
The function applies a bilateral filter to the input image using the OpenCV `bilateralFilter` function. The bilateral filter is a non-linear, edge-preserving smoothing filter that considers both spatial and intensity differences between pixels. The parameters used for this filter are set to 9 for the diameter of the pixel neighborhood, and 75 for both the color and spatial sigma values. These parameters control the extent of filtering in terms of pixel proximity and color similarity. The resulting filtered image is then returned as the output of the function.

In [ ]:
def apply_bilateral_filter(image):
    # bilateral filter (explain params)
    filtered_image = cv2.bilateralFilter(image, 9, 75, 75)
    return filtered_image